In [17]:
import pandas as pd
import numpy as np

In [128]:
orders = pd.read_excel('orders-test.xlsx')
graphic = pd.read_excel('graphic-test.xlsx')

In [129]:
orders

,Пункт отгрузки,КССС мат,"осталось произвести, тн"
0,ВНП,11466,3.345
1,ВНП,11466,3.540
2,ВНП,11466,1.860
3,ВНП,11466,13.845
4,ВНП,11466,4.320
5,ВНП,11466,6.480
6,ВНП,11466,2.010
7,ВНП,11466,10.080
8,ВНП,11466,15.000
9,ВНП,11466,15.000


In [130]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ВНП,11466,2022-12-17,5
1,ВНП,11466,2022-12-18,5
2,ВНП,11466,2022-12-19,5
3,ВНП,11466,2022-12-20,5
4,ВНП,11466,2022-12-21,5


In [131]:
# graphic.loc[(graphic['Пункт отгрузки']=='ВНП') & (graphic['Код КССС']==11466), 'Объем_фасовки, т'] = 0

In [132]:
# graphic['Объем_фасовки, т'] = 0

In [133]:
graphic[(graphic['Пункт отгрузки']=='ВНП') & (graphic['Код КССС']==11466)].empty

False

In [134]:
def data_shipment(city, code, required_to_make):
    # проверяем есть ли вообще в списке graphic
    if graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].empty:
        print(required_to_make)
        
        return np.nan
    # если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        print(required_to_make)
        return np.nan
    # иначе если есть и объема в графике достаточно, то мы    
    elif graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0] >= required_to_make:    
        print(str(required_to_make) + ' объема в графике достаточно')
        
        # вычитаем из объема
        # добавить определенное дату, так как будет удалять все значения где код и пункт совпадает
        data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                    (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[0]),'Дата_производства'].iloc[0]
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & (graphic['Дата_производства']==data_of_making),
                    'Объем_фасовки, т'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0]
        print(graphic)
        return data
    # иначе если недостаточно объема чем требуется ...
    else:
        print(str(required_to_make ) + 'else')
        # если возвращается только одна дата, возвращает -
        #graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0
        if len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) == 1:
            print(str(required_to_make ) + ' если возвращается только одна дата')
            graphic.loc[(graphic['Пункт отгрузки']=='ВНП') & (graphic['Код КССС']==11466), 'Объем_фасовки, т'] = 0
            return np.nan
        # иначе если возращается несколько дат
        elif len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) > 1:
            print(str(required_to_make ) + ' если возвращается несколько дат')
            print(graphic)
            # вводим переменную длины, чтобы не выйти за пределы
            len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
            # вводим счетчик для того чтобы присвоить правильную дату
            i = 0
            # создаем накопительную переменную 
            storage_var = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0]
            list_of_dates_to_be_deleted = []

            # пока объем фасовки меньше, чем требуется
            while storage_var<required_to_make and len_of_dates>1:
                len_of_dates-=1
                # переходим к следующей
                i+=1
                # и прибавляем к накопительной переменной
                storage_var+= graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                list_of_dates_to_be_deleted.append(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i-1]),
                                             'Дата_производства'].iloc[0])                    
            #если требуемый объем не накопился то возвращает -
            if storage_var<required_to_make:
                print(str(required_to_make ) + ' меньше чем требуется')
                return np.nan
            # иначе
            else:
                print(str(required_to_make ) + ' больше чем требуется')
                #нужно обнулить или удалить предыдущие даты, которые схлопнулись
                for j in (list_of_dates_to_be_deleted):
                    graphic.drop(graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                           (graphic['Дата_производства']==j)].index, inplace=True)
                    i-=1
                data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i]),
                                             'Дата_производства'].iloc[0]
        
            # вычитаем
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                        (graphic['Дата_производства']==data_of_making), 'Объем_фасовки, т'] = storage_var-required_to_make
 
            data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[i] 
            return data

In [135]:
orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

3.345 объема в графике достаточно
  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП     11466        2022-12-17             1.655
1            ВНП     11466        2022-12-18             5.000
2            ВНП     11466        2022-12-19             5.000
3            ВНП     11466        2022-12-20             5.000
4            ВНП     11466        2022-12-21             5.000
3.54else
3.54 если возвращается несколько дат
  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП     11466        2022-12-17             1.655
1            ВНП     11466        2022-12-18             5.000
2            ВНП     11466        2022-12-19             5.000
3            ВНП     11466        2022-12-20             5.000
4            ВНП     11466        2022-12-21             5.000
3.54 больше чем требуется
1.86 объема в графике достаточно
  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
1            ВНП     11466        2022-12-

In [136]:
print(graphic)

  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
4            ВНП     11466        2022-12-21               0.0


In [137]:
orders

,Пункт отгрузки,КССС мат,"осталось произвести, тн",дата производства
0,ВНП,11466,3.345,2022-12-17
1,ВНП,11466,3.540,2022-12-18
2,ВНП,11466,1.860,2022-12-18
3,ВНП,11466,13.845,2022-12-21
4,ВНП,11466,4.320,NaN
5,ВНП,11466,6.480,NaN
6,ВНП,11466,2.010,NaN
7,ВНП,11466,10.080,NaN
8,ВНП,11466,15.000,NaN
9,ВНП,11466,15.000,NaN


In [52]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
4,ВНП,11466,2022-12-21,0.4


In [712]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Объем_фасовки, т']

Series([], Name: Объем_фасовки, т, dtype: int64)

In [713]:
graphic[(graphic['Пункт отгрузки']=="ПНОС") & (graphic['Код КССС']==20746)].empty

True

In [714]:
##делать колонку с тем, сколько не хватает еще бессмысленно, так как продукт может далее распределиться по другим заказам, 
#для которых хватает и тогда число не будет иметь значение

In [715]:
# проверить если не накопилась дважды или трижды